<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
colab에서만 사용하는 코드
"""

# drive mount
from google.colab import drive
drive.mount('/content/drive')

# import_ipynb module 설치
!pip install import_ipynb

# 경로이동
%cd /content/drive/MyDrive/team_malmungchi/colab/speaker_verification/code
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/.shortcut-targets-by-id/1rEYIox5uYM9uP2spDttCYCqtt2GvSbn1/2022 기술 혁신 실험/colab/speaker_verification/code
batcher.ipynb	   eval_metrics.ipynb  network.ipynb
constants.ipynb    fitter.ipynb        preprocess.ipynb
development.ipynb  loss.ipynb	       validation.ipynb


In [2]:
import os
import sys
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
import pickle

#sys.path.append("/content/drive/MyDrive/team_malmungchi/colab/speaker_verification/code")
import import_ipynb
from constants import CHECKPOINTS_DIR, NUM_FRAME, NUM_FBANK
import batcher
import network
import loss
import fitter
from validation import validate

tf.config.run_functions_eagerly(True)

importing Jupyter notebook from constants.ipynb
importing Jupyter notebook from batcher.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from eval_metrics.ipynb
importing Jupyter notebook from loss.ipynb
importing Jupyter notebook from fitter.ipynb
importing Jupyter notebook from validation.ipynb


In [5]:
 def train(batcher_name, model_name, loss_name, checkpoint_dir=None, initial_epoch=0, max_epoch=0):
    

    # Batcher 로드 (train dataset 로드)
    Batcher = batcher.get_batcher(batcher_name)
    num_speaker= Batcher.num_speaker


    # test dataset 로드
    test_dataset_path = '../data/validation_batcher/64.npy'  # 일단 임시로 validation과 똑같다.
    with open(test_dataset_path,"rb") as f: test_X,_ = pickle.load(f)


    # optimizer, loss 객체 생성
    optimizer = tf.optimizers.Adam(learning_rate=0.001)
    Loss = loss.get_loss(loss_name)
    
    
    # 모델 생성
    Model = network.get_network(model_name, num_speaker)
    Model.summary()
    #Model.compile(optimizer='adam', loss=Loss, metrics=['accuracy']) # 이것도 metric이 경우마다 달라질 수 있겠다. fitter 안으로 넣자.
    Model.compile(optimizer='adam', loss=Loss) # simMat은 accuracy가 없음


    # 체크포인트 경로 생성
    if not checkpoint_dir:
      checkpoint_dir = f'../model/{model_name}-{batcher_name}-{loss_name}'
      if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)
    # 체크포인트 로드
    ckpts = os.listdir(checkpoint_dir)
    if ckpts:
      ckpts.sort(key=lambda x:float(x.split('-')[-1][:-5]))
      best_ckpt = ckpts[0]
      initial_epoch = int(best_ckpt.split('-')[0])
      Model.load_weights(checkpoint_dir+'/'+best_ckpt)
      print(f'\nload exist checkpoint that epoch:{initial_epoch}, EER:'+best_ckpt.split('-')[-1][:-5])
    # 체크포인트 객체 생성
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir+'/{epoch:05d}-{val_eer:.4f}.hdf5', monitor='val_eer', mode='min', save_best_only=True)

    
    # reduce_lr 객체 생성
    # reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.0001, verbose=1)


    
    # train
    fitter.fit(Model, Batcher, test_X, loss, initial_epoch, max_epoch, checkpoint)

    
    # validation
    eer = validate(Model)
    print(eer)
    # eer함수가 단위가 1이 max인듯. (eer=0.13->eer=13%)
    
    

In [6]:
#train('naive_batcher', 'naive_model', 'cross_entropy')

#train('simMat_batcher', 'naive_model', 'simMat_loss')

train('simMat_online_batcher', 'naive_model', 'simMat_loss', max_epoch=10)

load exist data
Model: "custom_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 4/10
10/10 [==============================] - 14s 271ms/step - loss: 33.0420 - val_eer: 0.3141
Epoch 5/10
10/10 [==============================] - 2s 170ms/step - loss: 33.8840 - val_eer: 0.3125
Epoch 6/10
10/10 [==============================] - 2s 216ms/step - loss: 39.1829 - val_eer: 0.3648
Epoch 7/10
10/10 [==============================] - 2s 151ms/step - loss: 39.6601 - val_eer: 0.3305
Epoch 8/10
10/10 [==============================] - 1s 135ms/step - loss: 37.7769 - val_eer: 0.3195
Epoch 9/10
10/10 [==============================] - 1s 133ms/step - loss: 37.5286 - val_eer: 0.3328
Epoch 10/10
10/10 [==============================] - 1s 156ms/step - loss: 35.5526 - val_eer: 0.3039
0.30390625
